# RTP Update

## Setup

In [1]:
# import packages
import pandas as pd
import pathlib
from pathlib import Path
import os
import arcpy
from utils import *
import numpy as np
import pickle

# pandas options
pd.options.mode.copy_on_write = True
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
pd.options.display.max_rows    = 999

# my workspace 
workspace = r"C:\Users\mbindl\Desktop\Workspace.gdb"
# current working directory
local_path = pathlib.Path().absolute()
# set data path as a subfolder of the current working directory TravelDemandModel\2022\
data_dir = local_path.parents[0] / 'data'
# folder to save processed data
out_dir  = local_path.parents[0] / 'data/processed_data'
# workspace gdb for stuff that doesnt work in memory
# gdb = os.path.join(local_path,'Workspace.gdb')
gdb = workspace
# set environement workspace to in memory 
arcpy.env.workspace = 'memory'
# # clear memory workspace
# arcpy.management.Delete('memory')

# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)

# get parcels from the database
# network path to connection files
filePath = "F:/GIS/PARCELUPDATE/Workspace/"
# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")
sdeEdit    = os.path.join(filePath, "Edit.sde")

# schema for the final output
final_schema = ['APN', 'Residential_Units', 'TouristAccommodation_Units', 'CommercialFloorArea_SqFt',
                'ZONING_ID', 'EXISTING_LANDUSE', 'COUNTY', 'JURISDICTION', 'OWNERSHIP_TYPE',
                'IPES_SCORE', 'VHR', 'BLOCK_GROUP', 'TAZ', 'RETIRED', 
                'JURISDICTION', 'COUNTY', 'OWNERSHIP_TYPE','EXISTING_LANDUSE',  
                'WITHIN_BONUS_UNIT_BNDRY', 'WITHIN_TRPA_BNDY',
                'MAX_RESIDENTIAL_UNITS', 'MAX_COMMERCIAL_FLOOR_AREA', 'MAX_TAU_UNITS',
                'PARCEL_ACRES', 'PARCEL_SQFT', 'SHAPE']

# Pickle variables
# part 1 - spatial join categories, occupancy rates, and parcels
parcel_pickle_part1    = data_dir / 'parcel_pickle1.pkl'


In [ ]:
# parcel development layer polygons
parcel_db = Path(sdeEdit) / "SDE.Parcel\\SDE.Parcel_History_Attributed"
# query 2022 rows
sdf_units = pd.DataFrame.spatial.from_featureclass(parcel_db)
sdf_units = sdf_units.loc[sdf_units['YEAR'] == 2022]
sdf_units.spatial.sr = sr

In [ ]:
sdf_units.CommercialFloorArea_SqFt.sum()

In [5]:
dfIPES     = pd.read_json("https://laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfRetired  = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")



In [ ]:
def get_conn(db):
    # Get database user and password from environment variables on machine running script
    db_user             = os.environ.get('DB_USER')
    db_password         = os.environ.get('DB_PASSWORD')

    # driver is the ODBC driver for SQL Server
    driver              = 'ODBC Driver 17 for SQL Server'
    # server names are
    sql_12              = 'sql12'
    sql_14              = 'sql14'
    # make it case insensitive
    db = db.lower()
    # make sql database connection with pyodbc
    if db   == 'sde_tabular':
        connection_string = f"DRIVER={driver};SERVER={sql_12};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    elif db == 'tahoebmpsde':
        connection_string = f"DRIVER={driver};SERVER={sql_14};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    elif db == 'sde':
        connection_string = f"DRIVER={driver};SERVER={sql_12};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    # else return None
    else:
        engine = None
    # connection file to use in pd.read_sql
    return engine

In [ ]:
# zoning data
engine = get_conn('sde')
with engine.begin() as conn:
    df_uses = pd.read_sql("SELECT * FROM sde.SDE.PermissibleUses", conn)

# function to get where Zoningin_ID Use_Type = Multi-Family and Density
def get_mf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Multiple Family Dwelling
    df = df.loc[df['Use_Type'] == 'Multiple Family Dwelling']
    return df[columns_to_keep]

def get_sf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Single Family Dwelling
    df = df.loc[df['Use_Type'] == 'Single Family Dwelling']
    return df[columns_to_keep]

def get_sf_only_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Single Family Dwelling and not Multiple Family Dwelling
    dfMF = get_mf_zones(df)
    dfSF = get_sf_zones(df)
    df = dfSF.loc[~dfSF['Zoning_ID'].isin(dfMF['Zoning_ID'])]
    return df[columns_to_keep]


dfMF = get_mf_zones(df_uses)
dfMF.Use_Type.value_counts()

dfSF = get_sf_zones(df_uses)
dfSF.Use_Type.value_counts()


dfSF_only = get_sf_only_zones(df_uses)
dfSF_only.Use_Type.value_counts()

In [ ]:
# parcel development layer polygons
parcel_db = Path(sdeEdit) / "SDE.Parcel\\SDE.Parcel_History_Attributed"
# query 2022 rows
sdf_units = pd.DataFrame.spatial.from_featureclass(parcel_db)
sdf_units = sdf_units.loc[sdf_units['YEAR'] == 2022]
sdf_units.spatial.sr = sr

### Forecast Formetted Table

In [ ]:
pathCSV = data_dir / "RegionalTransportationPlan/2023/data/Forecasts_Table1.csv"
print(pathCSV)

In [12]:
from great_tables import *
import pandas as pd

forecast = pd.read_csv(data_dir / "Forecasts_Table1.csv")
# drop notes column 
forecast.drop(columns=['Notes'], inplace=True)
# change column names
forecast.rename(columns={'Change by 2050': 'Change(#)', 'Percent Change': 'Change(%)'}, inplace=True)

In [ ]:
# version of great tables
gt.__version__


In [ ]:
from great_tables import GT, style, loc
from great_tables.data import gtcars

(
    GT(
        gtcars[["mfr", "model", "hp", "trq", "msrp"]].head(5),
        rowname_col="model",
        groupname_col="mfr"
    )
    .tab_stubhead(label="car")
    .tab_style(
        style=[
            style.text(color="crimson", weight="bold"),
            style.fill(color="lightgray")
        ],
        locations=loc.row_groups()
    )
    .fmt_integer(columns=["hp", "trq"])
    .fmt_currency(columns="msrp", decimals=0)
)

In [ ]:
forecast

In [14]:
GT(forecast).tab_header(title="Table 1. Forecast Data Summary").tab_spanner(
    label="", columns=['Category', 'Variable','Base Year 2022',  'Forecast 2050', 'Change(#)', 'Change(%)']).tab_stub(
        rowname_col='Variable', groupname_col='Category').tab_style(
            style=style.fill(color="aliceblue"), locations=loc.body()).save("forecast.jpeg")


### Transit Stacked Bar

In [ ]:
# get data for transit ridership
def get_data_transit():
    url = "https://www.laketahoeinfo.org/WebServices/GetTransitMonitoringData/CSV/e17aeb86-85e3-4260-83fd-a2b32501c476"

    dfTransit = pd.read_csv(url)
    dfTransit['Month'] = pd.to_datetime(dfTransit['Month'])
    dfTransit['Month'] = dfTransit['Month'].dt.strftime('%Y-%m')
    # filter out rows where RouteType is not Paratransit, Commuter, or Seasonal Fixed
    df = dfTransit.loc[~dfTransit['RouteType'].isin(['Paratransit', 'Commuter', 'Seasonal Fixed Route'])]
    # df = dfTransit.loc[dfTransit['RouteType'] != 'Paratransit']

    # replace transit operator values with abreviations
    df['TransitOperator'] = df['TransitOperator'].replace(
        ['Tahoe Transportation District',
       'Tahoe Truckee Area Regional Transit',
       'South Shore Transportation Management Association'],
       ["TTD", "TART", "SSTMA"])
    # route name = route type + transit operator
    df['RouteName'] = df['RouteType'] + ' - ' + df['TransitOperator']
    # group by RouteType, TransitOperator, and Month with sum of MonthlyRidership
    df = df.groupby(['RouteName', 'Month'])['MonthlyRidership'].sum().reset_index()
    # rename columns to Date, Name, Ridership
    df.rename(columns={'Month':'Date', 'RouteName':'Name', 'MonthlyRidership':'Ridership'}, inplace=True)
    # sort by Date
    df = df.sort_values('Date')
    return df

# html/3.3.a_Transit_Ridership.html
def plot_transit(df):
    trendline(
        df,
        path_html="html/3.3.a_Transit_Ridership.html",
        div_id="3.3.a_Transit_Ridership",
        x="Date",
        y="Ridership",
        color="Name",
        color_sequence=["#023f64", "#7ebfb5", "#a48352", "#FC9A62"],
        sort="Date",
        orders=None,
        x_title="Date",
        y_title="Ridership",
        markers=True,
        hover_data=None,
        tickvals=None,
        ticktext=None,
        tickangle=None,
        hovermode="x unified",
        format=",.0f",
        custom_data=["Name"],
        hovertemplate="<br>".join([
            "<b>%{y:,.0f}</b> riders on",
            "<i>%{customdata[0]}</i> lines"
                ])+"<extra></extra>",
        additional_formatting = dict(
                                    title = "Transit Ridership",
                                    margin=dict(t=20),
                                    legend=dict(
                                        # title="Transit Ridership",
                                        orientation="h",
                                        entrywidth=120,
                                        yanchor="bottom",
                                        y=1.05,
                                        xanchor="right",
                                        x=0.95,
                                    ))
    )